In [1]:
import logging
import hydra
import pandas as pd

from collections import defaultdict
from tqdm import tqdm 

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colormaps
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.patches import Ellipse

from erc import drawing_ellipse, split_df_by_gender


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset._target_=erc.datasets.KEMDy19Dataset"})
# select 1-fold 
# cfg.dataset.mode = "train"
train_dataset = hydra.utils.instantiate(cfg.dataset)

# cfg.dataset.mode = "valid"
# cfg.dataset.validation_fold = 0
# valid_dataset = hydra.utils.instantiate(cfg.dataset)

: 

: 

In [ ]:
# torch --> huggingface 
from datasets import Dataset

ds = Dataset.from_dict()

In [ ]:
from transformers import AutoTokenizer, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
def map_to_array(batch, processor):
    speech = processor(batch["wav"],
                          sampling_rate=batch["sampling_rate"],
                          return_tensors="pt",
                          return_attention_mask = False)['input_values']
                          
    batch["wav"] = speech
    return batch

In [ ]:
processor

In [ ]:
test_ds = train_dataset.map(map_to_array, processor)

In [ ]:
train_dataset[0]

In [ ]:

import torch
import torch.nn as nn 
import torch.nn.functional as F
from transformers import AdamW
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from jiwer import wer # wer metircs 
from transformers import Wav2Vec2Processor
device = torch.device('cuda:1')




In [ ]:
# Pre-training Scheme ... 
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

pretrain_str = "kresnik/wav2vec2-large-xlsr-korean"
# pretrain_str = "w11wo/wav2vec2-xls-r-300m-korean"

processor= Wav2Vec2Processor.from_pretrained(pretrain_str)
pretrained_model = Wav2Vec2ForSequenceClassification.from_pretrained(
    # "wav2vec2-xls-r-300m-korean",
    pretrain_str,
    num_labels=7
    )

In [ ]:
import erc

erc.utils.count_parameters(pretrained_model)

In [ ]:
model = pretrained_model.to(device)
# criterion = nn.MultiLabelSoftMarginLoss()
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr = 1e-5,  eps = 1e-8)

In [ ]:
import torch 
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size= 2)


from accelerate import Accelerator
accelerator = Accelerator()

model, optimizer, training_dataloader = accelerator.prepare(
     model, optimizer, train_loader)


In [ ]:

total_loss = 0
train_acc_sum = 0
train_loss = []
for step, batch in enumerate(train_loader): 
    optimizer.zero_grad()
    labels = (batch['emotion']).to(device)
    input_values = processor(batch["wav"],
                             sampling_rate=16000,
                             return_tensors="pt",
                             return_attention_mask = False)['input_values'].squeeze()
    inputs = {"input_values":input_values,
              "attention_mask":batch['wav_mask'],
    }
    inputs = {key: inputs[key].to(device) for key in inputs}


    logits = model(**inputs).logits

    
    # outputs = torch.argmax(logits, dim=-1)
    # print(logi)

    loss = criterion(logits, labels.long())
    total_loss += loss.item()
    train_loss.append(total_loss/(step+1))
    # print(loss.item())
    loss.backward()
    optimizer.step()

avg_train_loss = total_loss / len(train_loader)
print(f'  Average training loss: {avg_train_loss:.2f}')



In [ ]:
print(logits.shape)
print(labels.shape)
print(labels.dtype)
print(logits.dtype)


In [ ]:
batch

In [ ]:
logits.shape

In [ ]:
model.config.id2label

In [ ]:
torch.argmax(logits, dim=1)


In [ ]:
predicted_class_ids

In [ ]:
predicted_label = model.config.id2label[predicted_class_ids]

In [ ]:
predicted_label

In [ ]:


class EmotionClassfierWithAudio(nn.Module):
    def __init__(self,pretrained_model, n_classes=7):
        super().__init__()
        self.pretrianed_model = pretrained_model
        # for p in self.pretrianed_model.parameters():
            # p.requires_grad = False

        self.fc = nn.Linear(1025,n_classes)

    def forward(self, x):
        x = F.adaptive_avg_pool3d(self.pretrianed_model(x).logits , axis=1).squeeze()# mean of 355 // logit output shape: B, Seq(355), 1025
        x = self.fc(x)
        return x

In [ ]:
# !pip install jiwer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# import soundfile as sf
import torch
from jiwer import wer # wer metircs 
from transformers import Wav2Vec2Processor
batch = train_dataset[2]


processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
inputs = processor(batch["wav"], sampling_rate=16000, return_tensors="pt", return_attention_mask = False,
                #    padding="longest"
                   )

In [ ]:
inputs.keys()

In [ ]:
inputs['input_values']

In [ ]:
batch.keys()

In [ ]:
batch['wav']

In [ ]:
batch['wav_mask']

In [ ]:
batch['wav_mask'].shape

In [ ]:
inputs['attention_mask'].squeeze()


In [ ]:
inputs['attention_mask'].shape

In [ ]:



model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

input_values = inputs.input_values
print(input_values.shape)
with torch.no_grad():
    logits = model(input_values.type(torch.FloatTensor).to("cuda")).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(batch['txt'])
print(transcription)
print("WER:", wer(batch['txt'], transcription))

In [ ]:
logits.shape

In [ ]:
logits[:,-1].shape

In [ ]:
logits.shape

In [ ]:
logits.mean(axis=1).shape

In [ ]:
logits[:,-1]

In [ ]:
logits

In [ ]:
logits

In [ ]:
model(input_values.type(torch.FloatTensor).to("cuda"))

In [ ]:
predicted_ids